In [1]:
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time

#Время начала расчёта
start_time = time.time()

#Основные константы
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.9
KPD=0.99

#???
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'

#Импорт таблиц номинального режима и частичной нагрузки
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#???
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)


# ######################################################################
# #Связвка высокого давления
# start_time = time.time()
# it=50
# for j in range(it):
#     for i in range(it):
#         PEVD = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',       KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
#         gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
#         water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
#         IVD  = cotel.vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
#         gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
#         water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
#         water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
#         Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
#         Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
#         print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas))
#         if  abs((Qgas-Qwat)/Qgas) < calctolerance:
#             break;
#     EVD = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',               KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
#     gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
#     water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]
    
#     Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
#     Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
#     print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1))
#     if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
#         break;
# print('Степень сухости пара в ЭВД: ',water.p_h(water_streams.at['EVD-IVD','P'],water_streams.at['EVD-IVD','H'])['Q'])
# print("ВД:--- %s сек. ---" % round((time.time() - start_time)))
# ######################################################################



######################################################################
#Связка низкого давления
it=50
for j in range(it):
    #Связка ППНД+ИНД
    for i in range(it):
        #Расчёт ППНД
        PPND = cotel.heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
        water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
        #Расчёт ИНД
        IND  = cotel.vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',      KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
        water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
        #Переопределение расхода пара
        water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
        Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
        Qwat = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['GPK-IND','H'])
        #print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas))
        if  abs((Qgas-Qwat)/Qgas) < calctolerance:
            break;
    #ПЭН
    water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
    PEN=nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams).calc()
    water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]
    
    
    #Расчёт расхода в ГПК
    water_streams.loc['X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
    water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
    #Расчёт ГПК
    GPK = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
    gas_streams.loc['GPK-out','T':'G']   = [GPK[0],GPK[1],GPK[2],GPK[3]]
    water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]#!!!!!!
    
    
    #Q ГПК
#     Qgas = KPD*gas_streams.at['GPK-out','G']*(gas_streams.at['IND-GPK','H']-gas_streams.at['GPK-out','H'])
#     Qwat = water_streams.at['X-GPK','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])
#     print('GPK',Qgas-Qwat)
#         #Q ППНД
#     Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['PPND-IND','H'])
#     Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])
#     print('PPND',Qgas-Qwat)
#         #Q ИНД
#     Qgas = KPD*gas_streams.at['PPND-IND','G']*(gas_streams.at['PPND-IND','H']-gas_streams.at['IND-GPK','H'])
#     Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])
#     print('IND',Qgas-Qwat)
#     print(PPND[8],IND[8],GPK[8])
    
        
    Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
#     Qwat1 = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['X-GPK','H'])
    Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
    #print(water_streams.at['PPND-TURB','G']*(water_streams.at['BND-PEN','H']-water_streams.at['X-GPK','H']))
    print('dQ/Q ППНД+ИНД+ГПК',((Qgas1-Qwat1)))
#     print(abs((Qgas1-Qwat1)/Qgas1))
    if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
        break;
print('Степень сухости пара в ГПК: ',water.p_h(water_streams.at['GPK-IND','P'],water_streams.at['GPK-IND','H'])['Q'])
print("НД:--- %s сек. ---" % round((time.time() - start_time)))
######################################################################






dQ/Q ППНД+ИНД+ГПК -285.19619684887584
dQ/Q ППНД+ИНД+ГПК -316.64665609263466
dQ/Q ППНД+ИНД+ГПК -339.8584299796348
dQ/Q ППНД+ИНД+ГПК -354.4449825486372
dQ/Q ППНД+ИНД+ГПК -363.2219738873973
dQ/Q ППНД+ИНД+ГПК -369.34295439129346
dQ/Q ППНД+ИНД+ГПК -373.20026251517993
dQ/Q ППНД+ИНД+ГПК -375.67924438432965
dQ/Q ППНД+ИНД+ГПК -377.2677513672388
dQ/Q ППНД+ИНД+ГПК -378.2862870861136
dQ/Q ППНД+ИНД+ГПК -378.939374765323
dQ/Q ППНД+ИНД+ГПК -379.35816221675486
dQ/Q ППНД+ИНД+ГПК -379.6267148862098
dQ/Q ППНД+ИНД+ГПК -379.79893122412614
dQ/Q ППНД+ИНД+ГПК -379.9093708017317
dQ/Q ППНД+ИНД+ГПК -379.9801945241634
dQ/Q ППНД+ИНД+ГПК -380.02561326278374
dQ/Q ППНД+ИНД+ГПК -380.054740040403
dQ/Q ППНД+ИНД+ГПК -380.0734189544455
dQ/Q ППНД+ИНД+ГПК -380.0853976708022
dQ/Q ППНД+ИНД+ГПК -380.0930796045868
dQ/Q ППНД+ИНД+ГПК -380.0980060069269
dQ/Q ППНД+ИНД+ГПК -380.1011652933812
dQ/Q ППНД+ИНД+ГПК -380.1031913413317
dQ/Q ППНД+ИНД+ГПК -380.1044906526513
dQ/Q ППНД+ИНД+ГПК -380.1053238874447
dQ/Q ППНД+ИНД+ГПК -380.105858255

KeyboardInterrupt: 

In [ ]:
gas_streams

In [ ]:
water_streams

In [ ]:
26322.768635161203
